In [92]:
#import geopandas as gpd
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from math import radians, cos, sin, asin, sqrt
import easygui
import os
from pyproj import Proj, transform
import numpy as np

In [2]:
def make_colormap(seq):
    """Return a LinearSegmentedColormap
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
    and in the interval (0,1).
    """
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

In [21]:
def proj_utm11(lon,lat):
    outProj = Proj(init='epsg:32611')
    inProj = Proj(init='epsg:4326')
    x2=[];y2=[]
    for i in range(0,len(lon)):
        x_tmp,y_tmp = transform(inProj,outProj,lon[i],lat[i])
        x2.append(x_tmp)
        y2.append(y_tmp)
    return x2,y2

In [3]:
readfile = easygui.fileopenbox(default='/home/martin/nevada/Data/',filetypes=['*.csv'])

In [4]:
data_df=pd.read_csv(readfile,sep=',', skiprows=2)
ch_00=data_df.columns.get_loc("Ch_00")
roi=[10,1022]

In [101]:
data_df['ch_sum']=data_df.ix[:,ch_00+roi[0]:ch_00+roi[1]].sum(axis=1)
df1 = data_df.ix[:, ['Long','Lat','Alt[m]','ch_sum']]
df1=df1[(df1.Lat !=0) & (df1.Long !=0)]

In [208]:
from osgeo import gdal
geofile=easygui.fileopenbox(default='/home/martin/nevada/Data/Nevada',filetypes=['*.tif'])

In [209]:
ds = gdal.Open(geofile)
r_width = ds.RasterXSize
r_height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + r_width*gt[4] + r_height*gt[5] 
maxx = gt[0] + r_width*gt[1] + r_height*gt[2]
maxy = gt[3] 

In [71]:
lon,lat=df1.Long.tolist(),df1.Lat.tolist()

In [210]:
print r_width

3480


In [103]:

#x,y=proj_utm11(x_tmp[i],y_tmp[i])
outProj = Proj(init='epsg:32611')
inProj = Proj(init='epsg:4326')
x2=[];y2=[]
for i in range(len(lon)):
    try:
        x_tmp,y_tmp = transform(inProj,outProj,lon[i],lat[i])
    except:
        print i
        
    x2.append(x_tmp)
    y2.append(y_tmp)
#return x2,y2

In [104]:
df1['x']=x2
df1['y']=y2

In [82]:
p_red=df1[(df1.x < maxx) & (df1.x > minx) & (df1.y > miny) & (df1.y < maxy)]
x=p_red.x.tolist();y=p_red.y.tolist();z=p_red.ch_sum.tolist()

In [86]:
area=mpimg.imread(geofile)

In [88]:
plt.imshow(area)
plt.plot(x,y,'.')
plt.show()

In [81]:
px

[]

In [ ]:
m = Basemap(resolution='i',
projection='tmerc',width=,height=lat_0=,lon_0=)

In [93]:
band = ds.GetRasterBand(1) #bands start at one
a = band.ReadAsArray().astype(np.float)

In [94]:
plt.imshow(a)

In [95]:
plt.show()

In [115]:
x_cen=minx+(maxx-minx)/2.
y_cen=miny+(maxy-miny)/2.

In [97]:
def proj_latlong(utm11x,utm11y):
    inProj = Proj(init='epsg:32611')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj,outProj,utm11x,utm11y)
    return x2,y2

In [116]:
lon_cen,lat_cen=proj_latlong(x_cen,y_cen)

In [140]:
m = Basemap(resolution='i',projection='tmerc',width=r_width,height=r_height,lat_0=lat_cen,lon_0=lon_cen)

In [141]:
#p_red=df1[(df1.Long < r) & (df1.Long > l) & (df1.Lat > d) & (df1.Lat < u)]
p_red=df1[(df1.x < maxx) & (df1.x > minx) & (df1.y > miny) & (df1.y < maxy)]
x=p_red.Long.tolist();y=p_red.Lat.tolist();z=p_red.ch_sum.tolist()
#x=p_red.x.tolist();y=p_red.y.tolist();z=p_red.ch_sum.tolist()
xp,yp=m(x,y)

In [142]:
c = mcolors.ColorConverter().to_rgb
rvb = make_colormap(
    [c('blue'), c('green'), 0.11, c('green'),c('yellow'),0.30,c('yellow'), c('red'),0.90,c('red')])

rvb2 = make_colormap([c('blue'),0.2,c('blue'),c('green'),0.4,c('green'),c('yellow'),0.6,c('yellow'),c('red'),0.8,c('red')])

In [107]:
geofile

'/home/martin/nevada/Data/Nevada/Area3_image_utm11.tif'

In [143]:
#area=mpimg.imread("%s_latlon.tif" % geofile[0:-4])
area=mpimg.imread(geofile)
m.imshow(area,origin='upper', extent=[minx,maxx,miny,maxy])
m.scatter(xp, yp,s=40,c=z,edgecolor='none',cmap=rvb) 
#m.drawmapscale(lon=l+(r-l)/6,lat=d+(u-d)/15,lat0=0,lon0=0,length=dist,units='m',barstyle='fancy')
cb=m.colorbar(size='3%')
ax=cb.ax
ax.text(0,1.02,'Total counts',size=14)

plt.show()


In [137]:
r_width, r_height

(3480, 2177)

In [144]:
minx

583745.4768506744

In [145]:
maxx

589637.575221637

In [146]:
miny,maxy

(4098093.050156291, 4101778.9979015742)

In [152]:
max(x)

-116.00939890000001

In [148]:
max(xp)

903.7396997007401

In [153]:
lon_cen

-116.02515587213726

In [155]:
lat_cen

37.04181432471869

In [156]:
r,u =proj_latlong(maxx,maxy)
l,d =proj_latlong(minx,miny)
#cx,cy= proj_latlong(minx+width/2.,miny+height/2.)

In [157]:
cx,cy

(-116.03880184558874, 37.035124807547994)

In [205]:
#m = Basemap(llcrnrlon=l,llcrnrlat=d,urcrnrlon=r,urcrnrlat=u,resolution='i',
 #           projection='tmerc',width=r_width,height=r_height,lat_0=0,lon_0=-116)
m = Basemap(llcrnrx=minx,llcrnry=miny,urcrnrx=maxx,urcrnry=maxy,resolution='i',
           projection='tmerc',width=maxx-minx,height=maxy-miny,lat_0=lat_cen,lon_0=lon_cen)   

In [199]:
minx, maxx, miny,maxy

(583745.4768506744, 589637.575221637, 4098093.050156291, 4101778.9979015742)

In [203]:
r_width,r_height

(3480, 2177)

In [204]:
maxx-minx

5892.09837096266

In [206]:
p_red=df1[(df1.x < maxx) & (df1.x > minx) & (df1.y > miny) & (df1.y < maxy)]
x=p_red.Long.tolist();y=p_red.Lat.tolist();z=p_red.ch_sum.tolist()
#x=p_red.x.tolist();y=p_red.y.tolist();z=p_red.ch_sum.tolist()
xp,yp=m(x,y)

In [207]:
#area=mpimg.imread("%s_latlon.tif" % geofile[0:-4])
area=mpimg.imread(geofile)
m.imshow(area,origin='upper', extent=[minx,maxx,miny,maxy])
m.scatter(xp, yp,s=40,c=z,edgecolor='none',cmap=rvb) 
#m.drawmapscale(lon=l+(r-l)/6,lat=d+(u-d)/15,lat0=0,lon0=0,length=dist,units='m',barstyle='fancy')
cb=m.colorbar(size='3%')
ax=cb.ax
ax.text(0,1.02,'Total counts',size=14)

plt.show()

In [166]:
lat_cen, lon_cen

(37.04181432471869, -116.02515587213726)